In [4]:
import pandas as pd
df_new = pd.read_csv('../data/clean_data.csv', sep='\t')
df_new.dropna(subset=['events'], inplace=True)

In [5]:
documents = df_new['content'].tolist()
events = df_new['events'].tolist()

In [9]:
file1 = open("../data/documents_labeled.txt","w")
count = 0
for i in range(len(documents)):
#     if ' '.join(set(events[i].split())) == '股价下跌':
    file1.write(documents[i] + "\n")
    count += 1
    if count == 25000:
        break
file1.close()

print("finished extracting {} docutments".format(count))

finished extracting 25000 docutments


In [4]:
import jieba
import gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
def load_stopwords(stopwords_path):
    with open(stopwords_path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f]
    
def preprocess_data(corpus_path, stopwords):
    corpus = []
    with open(corpus_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = ''.join(line.split())
            corpus.append(' '.join([word for word in jieba.lcut(line.strip()) if word not in stopwords]))
    return corpus

In [6]:
stopwords_path = "../data/stop_words.txt"
documents_path = "../data/documents_股价上涨.txt"
stopwords = load_stopwords(stopwords_path)
documents = preprocess_data(documents_path, stopwords)

Building prefix dict from the default dictionary ...
2020-06-18 15:55:38,237 : DEBUG : Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/m3/4yh806w92fdgcn0bk16ql7nw0000gn/T/jieba.cache
2020-06-18 15:55:38,240 : DEBUG : Loading model from cache /var/folders/m3/4yh806w92fdgcn0bk16ql7nw0000gn/T/jieba.cache
Loading model cost 0.955 seconds.
2020-06-18 15:55:39,193 : DEBUG : Loading model cost 0.955 seconds.
Prefix dict has been built successfully.
2020-06-18 15:55:39,195 : DEBUG : Prefix dict has been built successfully.


In [46]:
corpuses = []
with open(documents_path, 'r', encoding='utf-8') as f:
    for line in f:
        line = ''.join(line.split())
        corpuses.append(line)    

In [7]:
import numpy as np
import math
BITS = 31
class simHash(object):
# 初始化，遍历文档（已分词），得到词汇表，并进行32位(能表示2^32种情况，完全足够)\
#hash编码和对应的idf值，将idf值作为其权重进行运算，分别存入两个字典(dict)
    def __init__(self, documents):
        f = documents
        dictHash = dict()
        dictWeight = dict()
        i = 0#hash编码
        lines = 0#记录文本数量,以计算idf值
        #遍历文本，进行hash编码和统计df词频(在多少篇文章出现过，而不是总词频，\
        #比如某个词在一个文本中出现三次也只算一次)
        for line in f:
            lines += 1
            temp = set(str(line).strip().split())#避免重复统计词频
            for item in temp:
                if item not in stopwords:
                    if item not in dictWeight:
                        dictWeight[item] = 1
                        dictHash[item] = i
                        i += 1
                    else:
                        dictWeight[item] += 1
        del i
        #hash编码转为array形式的二进制，方便计算
        for item in dictHash:
            L = list(bin(dictHash[item]))[2:]
            intL = [int(x) for x in L]
            for i in range(len(intL)):
                if intL[i] == 0:
                    intL[i] = -1
            intL = (BITS - len(intL))*[-1]+intL
            dictHash[item] = np.array(intL)
        #根据词频计算idf值
        for item in dictWeight:
            dictWeight[item] = math.log(lines/dictWeight[item])

        self.dictHash = dictHash
        self.dictWeight = dictWeight
        
    #根据词的hash对句子进行hash编码
    def senHash(self, sen):
        senHashCode = np.zeros(BITS)
        temp = sen.strip().split()
        for item in temp:
            senHashCode += self.dictHash[item]*self.dictWeight[item]
        for i in range(BITS):
            if senHashCode[i] > 0:
                senHashCode[i] = 1
            else:
                senHashCode[i] = 0
        return senHashCode

    #获取两个句子的Hamming distance，dis越小说明相似度越高
    def sen2senDis(self, sen1, sen2):
        temp1 = self.senHash(sen1)
        temp2 = self.senHash(sen2)
        Hamming = 0
        for i in range(BITS):
            if temp1[i] != temp2[i]:
                Hamming += 1
        return Hamming


In [8]:
simhash = simHash(documents)

In [33]:
print(simhash.dictHash['上涨'])
print(simhash.dictWeight['上涨'])

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  1 -1 -1 -1  1 -1 -1]
1.029340723324554


In [10]:
len(documents)

10763

In [24]:
minSimHash = 10000
minSimHashIndex = -1
for i in range(0, len(documents)):
    if i == 100:continue
    temp = simhash.sen2senDis(documents[100], documents[i])
    if temp < minSimHash:
        minSimHash = temp
        minSimHashIndex = i
        
print(minSimHash, minSimHashIndex)

0 571


In [25]:
print(documents[100])
print(documents[571])


格隆汇 月 日 丨 啤酒 板块 走强 珠江 啤酒 盘中 一度 涨停 现涨 青岛 啤酒 涨近 惠泉 啤酒 燕京啤酒 集体 走高 近期 餐饮行业 复工 进度 加快 有利于 酒类 行业 恢复 历史 经验 二季度 酒类 板块 收益 概率 酒类 板块 迎来 最佳 配置 期川财 证券 指出 中国 啤酒 行业 步入 高端 机遇期 未来 产品 结构 升级 业绩 增长 核心 驱动力 消费 场景 消费 偏好 角度 短期 疫情 影响 预计 啤酒 家庭 自饮 消费 占 提升 长期 利好 注重 产品品质 差异化 更优 头部 品牌
事件 具体内容 月 日二 十国集团 农业部长 视频会议 会后 发表 声明 十国集团 农业部长 承诺 紧密 合作 维护 全球 粮食安全 声明 重申 努力 确保 粮食 产品 农业 粮食 生产 必需 投入品 跨境 流动 重要性 努力 确保 农业 粮食 供应链 工人 健康 福利 流动性 公司 紧紧围绕 农业 食品 两大 产业 全球 资源 中国 市场 发展 理念 专注 公司 农资 粮食 贸易 乳业 肉牛 三大 业务 二级 市场走势 该股 今日 强势 上涨 后市 有望 冲高


In [23]:
simhash.sen2senDis(documents[0], documents[0])

0

In [40]:
def findsimilar(index, documents):
    minSimHash = 10000
    minSimHashIndex = -1
    sim0, sim1, sim2 = [], [], []
    for i in range(0, len(documents)):
        if i == index:continue
        temp = simhash.sen2senDis(documents[index], documents[i])
        if temp == 0 : sim0.append(i)
        if temp == 1 : sim1.append(i)
        if temp == 2 : sim2.append(i)
            
        if temp < minSimHash:
            minSimHash = temp
            minSimHashIndex = i
    print(minSimHash, minSimHashIndex)
    return sim0, sim1, sim2

In [41]:
sim0, sim1, sim2 = findsimilar(999, documents)

0 667


In [50]:
print(sim2)

[8, 100, 102, 170, 232, 340, 343, 477, 483, 509, 571, 682, 742, 780, 1127, 1178, 1196, 1244, 1346, 1385, 1388, 1450, 1485, 1512, 1548, 1569, 1667, 1703, 1748, 1814, 1824, 1839, 1844, 1990, 2017, 2024, 2086, 2128, 2169, 2256, 2262, 2308, 2354, 2439, 2442, 2463, 2473, 2493, 2585, 2610, 2749, 2790, 2848, 2960, 3103, 3107, 3122, 3127, 3156, 3177, 3230, 3274, 3292, 3366, 3506, 3585, 3649, 3703, 3746, 3751, 3923, 3949, 4078, 4239, 4261, 4297, 4389, 4401, 4426, 4487, 4490, 4519, 4522, 4539, 4540, 4541, 4561, 4584, 4616, 4666, 4728, 4817, 4829, 4933, 4982, 5111, 5144, 5147, 5170, 5171, 5212, 5275, 5353, 5377, 5441, 5474, 5481, 5483, 5493, 5498, 5594, 5640, 5647, 5658, 5661, 5720, 5736, 5766, 5813, 5907, 5932, 5997, 6051, 6073, 6162, 6178, 6343, 6349, 6385, 6444, 6504, 6514, 6538, 6560, 6568, 6575, 6613, 6641, 6675, 6678, 6690, 6699, 6714, 6726, 6824, 6908, 6913, 6915, 6943, 6959, 6981, 7031, 7045, 7150, 7155, 7175, 7180, 7182, 7196, 7228, 7251, 7255, 7261, 7263, 7269, 7276, 7279, 7306, 7326, 7

In [48]:
print(corpuses[999])
print(corpuses[667])
print(corpuses[2715])
print(corpuses[4438])
print(corpuses[4951])
print(corpuses[5473])


格隆汇4月23日丨国美系继续上涨国美通讯600898.SH再度封板本周暂四日三板累涨近40国美零售0493.HK涨逾3本周暂累涨逾34中关村涨近2国美通讯昨晚公布公司筹划重大资产重组拟通过现金出售的方式向北京美昊投资出售浙江德景电子100股权此举将有利于进一步提升上市公司质量此外上周末国美零售发公告称向拼多多溢价66.44发行2亿美元3年期可转债
新浪财经讯4月23日消息食品加工制造板块拉升截至发稿海欣食品拉升涨停龙大肉食盐津铺子克明面业金字火腿等跟涨责任编辑田原
以下是御银股份在北京时间4月16日1354分盘口异动快照4月16日御银股份盘中涨幅达5截至13点54分报4.66元成交9442.09万元换手率2.72注以上信息仅供参考不对您构成任何投资建议
事件具体内容中国电信中国移动中国联通联合举行线上发布会发布5G消息白皮书阐述了5G消息的核心理念明确了相关业务功能及技术需求提出了对5G消息生态建设的若干构想有业内人士称白皮书的发布有助于三大运营商之间的互联互通打通RCS服务壁垒加速RCS产业发展公司有虚拟运营商牌照和三大运营商有合作公司依靠自主研发的应用软件系统目前已发展成为国内通过SaaS模式为中小企业信息化建设提供软件应用及服务最主要的提供商之一是国际知名的为全球企业用户组织机构及个人用户提供互联网应用服务的提供商ASP二级市场走势该股今日强势上涨后市有望继续冲高
以下是沃特股份在北京时间4月13日1431分盘口异动快照4月13日14点31分沃特股份盘中打开涨停现报29.19元成交2.56亿元换手率18.48该股之前于14点30分涨停注以上信息仅供参考不对您构成任何投资建议
以五粮液为代表的中国白酒有着3000多年的酿造历史堪称世界最古老最具神秘特色的食品制造产业之一五粮液作为曾经的白酒老大在经历了塑化剂事件之后随着近年来新的管理层励精图治之后逐步回归到人们的视野中2019年从市场表现来看遭北上资金重点加仓的股票明显跑赢市场贵州茅台五粮液等更已翻倍遭北上资金抛弃的股票大多跑输市场平均作为港股通的重仓股价值投资的典范之一我们试着以后视镜的角度出发以2010年12月31日的收盘价买入100股五粮液回测下持有至今的收益率截止2020年4月9日五粮液的收盘价124.98元2010年12月31日收盘价34.63元100股成本3463元期间无拆股派息如下总分红760元目前

In [52]:
maxSimHash = 0
maxSimHashIndex = -1
for i in range(0, len(documents)):
    if i == 100:continue
    temp = simhash.sen2senDis(documents[100], documents[i])
    if temp > maxSimHashIndex:
        maxSimHash = temp
        maxSimHashIndex = i
        
print(maxSimHash, maxSimHashIndex)

11 249


In [55]:
print(documents[249])
print(documents[100])


合 锻 智能 北京 时间 月 日 1021 分盘口 异动 快照 月 日合 锻 智能 盘中 涨幅 达 点 分报 5.71 成交 4701.68 换手率 1.88 注 信息 仅供参考 投资 建议
格隆汇 月 日 丨 啤酒 板块 走强 珠江 啤酒 盘中 一度 涨停 现涨 青岛 啤酒 涨近 惠泉 啤酒 燕京啤酒 集体 走高 近期 餐饮行业 复工 进度 加快 有利于 酒类 行业 恢复 历史 经验 二季度 酒类 板块 收益 概率 酒类 板块 迎来 最佳 配置 期川财 证券 指出 中国 啤酒 行业 步入 高端 机遇期 未来 产品 结构 升级 业绩 增长 核心 驱动力 消费 场景 消费 偏好 角度 短期 疫情 影响 预计 啤酒 家庭 自饮 消费 占 提升 长期 利好 注重 产品品质 差异化 更优 头部 品牌
